In [2]:
# Options for all cells 
import pandas as pd
# change display setting of pandas
pd.set_option('display.notebook_repr_html', False)
# Setting the graphics
%matplotlib inline
# suppress all warnings (since anova gives a warning)
import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd
tips = pd.read_csv("Data/tips.csv")

## Statistical Modelling

There are a number of packages that contain functions for statistics and modelling in the Python standard library. We will use the **statsmodel** package which contains a large number of models and statistical tests, organised in several modules. Some of the more common model fitting functions can be found in the following table:

| Package/Module | Function | Model |
|----------------|----------|-------|
| `statsmodels.formula.api` | `ols`/`OLS` | Linear Regression by OLS (Ordinary Least Squares) |
| `statsmodels.formula.api` | `gls`/`GLS` | Linear Regression by GLS (Generalised Least Squares) |
| `statsmodels.api` | `GLM` | GLM (Generalised Linear Model) |
| `statsmodels.api` | `RLM` | RLM (Robust Linear Model) |
| `statsmodels.tsa.arima_model` | `ARIMA` | Time Series Analysis - ARIMA Model |
| `statsmodels.formula.api` | `phreg` | Survival Analysis - Cox Models |
| `statsmodels.api` | `stats.anova_lm` | ANOVA (Analysis of Variance)


For a full list of functionality, see the package's site [http://www.statsmodels.org/dev/index.html](http://www.statsmodels.org/dev/index.html).

Similar to modelling in R, the model is specified through a formula with a tilde (`~`) which typically has the response on the left-hand side and the independent variables on the right-hand side. The standard formula notation is the same as in R, e.g.,

| Python Formula | Model | Actual Model Formula |
|----------------|-------|----------------------|
| `y ~ x` | Y against X + an intercept | $y = a + bx + \epsilon$ |
| `y ~ x-1` | Y against X (no intercept) | $y = bx + \epsilon$ |
| `y ~ x+z` | Y against X and Z + an intercept | $y = a + bx + cz + \epsilon$ |
| `y ~ x:z` | Y against the X/Z interaction term | $y = a + bxz + \epsilon$ |

where in the actual model formulae above, $\epsilon$ is the error term.

In R, we typically provide a modelling function with the formula and the data and it returns a fitted model. Here, model specification and model fitting are two separate steps. Using the `ols` function, we specify a linear model for the tip as explained by the total amount of the bill.

In [2]:
import statsmodels.formula.api as smf

m1 = smf.ols(formula="tip ~ total_bill", data=tips)

The resulting model object has a `.fit` method which is then used to estimate the parameters of the model. For the fitted model object, further methods and attributes are available, e.g., a summary method (`.summary`) and the estimated parameters (`.params`).

In [3]:
mf1 = m1.fit()
print(mf1.summary())

                            OLS Regression Results                            
Dep. Variable:                    tip   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     203.4
Date:                Fri, 17 Aug 2018   Prob (F-statistic):           6.69e-34
Time:                        17:52:56   Log-Likelihood:                -350.54
No. Observations:                 244   AIC:                             705.1
Df Residuals:                     242   BIC:                             712.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9203      0.160      5.761      0.0

**Exercise**

Fit a linear model for `tip` as explained through `total_bill` and `smoker`.

In [4]:
# solutions
import statsmodels.formula.api as smf
model_s = smf.ols(formula="tip ~ total_bill + smoker", data=tips)
print(model_s.fit().summary())


                            OLS Regression Results                            
Dep. Variable:                    tip   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.455
Method:                 Least Squares   F-statistic:                     102.4
Date:                Fri, 17 Aug 2018   Prob (F-statistic):           6.57e-33
Time:                        17:52:57   Log-Likelihood:                -349.93
No. Observations:                 244   AIC:                             705.9
Df Residuals:                     241   BIC:                             716.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.9632      0.164      5.861

Alternatively, we can fit a model with an interaction between `total_bill` and `smoker`, and compare this to the simpler model via an ANOVA (analysis of variance).

In [8]:
import statsmodels.api as sm

mf2 = smf.ols(formula="tip ~ total_bill + smoker + total_bill:smoker", data=tips).fit() # or total_bill*smoker
#mf2.summary()
sm.stats.anova_lm(mf1, mf2)

/Users/hfrick/anaconda/envs/nb2docx/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/hfrick/anaconda/envs/nb2docx/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/hfrick/anaconda/envs/nb2docx/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,242.0,252.788744,0.0,NaN,NaN,NaN
1,240.0,229.809386,2.0,22.979358,11.999175,0.000011


**Warning : Python Warnings**

You may notice a warning appear when running the above function. This occurs due to the `NaN` values in our output; however, this function will always produce NaN values there so this is nothing to worry about.

Warnings appear red in Jupyter, but do not stop the computation of the function, so can *in some cases* be ignored.

## Machine Learning

While R has its strength in statistical modelling, Python shines in machine learning, in particular with the **scikit-learn** package.

### Getting Data in the Right Format

So far, we stored all our data in one `DataFrame`. The functions in **scikit-learn** for models and algorithms expect the features, or explanatory variables, and the target, or response, in two separate objects: The features are in a 2-dimensional array with one row per sample or observation and one column per feature or variable. The target is in an one-dimensional array.

We can convert a `DataFrame` using the `values` attribute. Here we will use the tips data to create an array of features and the one-dimensional array of the tips as our target.

In [9]:
import pandas as pd
tips_data = tips.drop("tip", axis=1).values
tips_target = tips["tip"].values

The example datasets in **scikit-learn** come in form of dictionaries which hold the two arrays for features and target along with the corresponding names. Here we will use the iris data which contains three different species of iris flowers as the target and four different measurements (length and width of the sepal and petal, respectively) as the features.

In [10]:
from sklearn import datasets 
iris = datasets.load_iris()
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In the following we will use the generic names `X` and `y` for the features and target, respectively. Python allows us to assign multiple objects at once as follows:

In [11]:
X, y = iris.data, iris.target

### Classification

Classification is common machine learning task so we will use this as example to illustrate a common workflow with **scikit-learn**. 

Similar to working with **statsmodel**, specifying a model, also called instantiating a model, is a separate step from fitting a model. Here we will first instantiate a K-nearest neighbor classifier and then fit it on the iris data.

In [12]:
from sklearn import neighbors

# create the model
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# fit the model
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

Predictions can be accessed with the `.predict` method for the model object.

In [13]:
knn.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
# What kind of iris has 3cm x 5cm sepal and 4cm x 2cm petal?
result = knn.predict([[3, 5, 4, 2],])

print(iris.target_names[result])

['versicolor']


### Model Validation

These predictions can be used to calculate the accuracy of the predictions. However, choosing a model based on its accuracy on the same data that was used to fit the model often leads to choosing a model too closely adapted to the data.

To avoid this over-fitting, the data is typically split into train and test data. We can easily do this with the `train_test_split` function and then train our classifier only on the training set.

In [15]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

Now we can calculate the accuracy on the test set.

In [16]:
from sklearn import metrics

y_pred = knn.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.9210526315789473

This is also available as a method, `.score`, for the model object.

In [17]:
knn.score(X_test, y_test)

0.9210526315789473

Cross-validation takes this idea a step further: The data is split into k folds and the model is fit k times, always leaving one of the folds as the test set.

In [18]:
from sklearn import model_selection

cv = model_selection.cross_val_score(neighbors.KNeighborsClassifier(5), X, y, cv=10)
cv.mean()

0.9666666666666668

The modules in **scikit-learn** are designed such that we can easily swap out one model for another.

**Exercise**

1. Use the `SVC` function from `sklearn.svm` to solve the iris classification problem with a support vector classifier. What is the accuracy on the training set?

Extension:

2. What is the average accuracy when using a 10-fold cross-validation?

In [19]:
# solution
from sklearn import svm 

svc = svm.SVC()
svc.fit(X_train, y_train)
knn.score(X_test, y_test)

cv = model_selection.cross_val_score(svm.SVC(), X, y, cv=10)
cv.mean()

0.9800000000000001